In [ ]:
from tensorflow.keras.models import Sequential
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications import ResNet50V2

In [ ]:
data_dir = os.path.join('/kaggle/input/rice-pistachio-and-grapevine-leaf-classification/train', 'train')
df = pd.read_csv("/kaggle/input/rice-pistachio-and-grapevine-leaf-classification/train.csv")

In [ ]:
def load_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, (384, 384))
    img = img/255.0

    return img, label

In [ ]:
class_names = df['TARGET'].unique()
class_to_int = {name: i for i, name in enumerate(class_names)}
int_to_class = {i: name for i, name in enumerate(class_names)}

In [ ]:
full_paths = [os.path.join(data_dir, fname) for fname in df['ID'].values]
label = df['TARGET'].map(class_to_int).values
dataset = tf.data.Dataset.from_tensor_slices((full_paths, label))
dataset = dataset.map(load_image)
dataset = dataset.shuffle(buffer_size=len(full_paths))

In [ ]:
BATCH_SIZE = 32
num_images = 6400
train_size = int(num_images * 0.8)
val_size = int(num_images * 0.2)
train = dataset.take(train_size)
val = dataset.skip(train_size).take(val_size)

train = train.batch(BATCH_SIZE)
val = val.batch(BATCH_SIZE)

In [ ]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2)
], name="data_augmentation")

In [ ]:
pre_trained_model = ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(384,384,3),
    pooling='avg'
)

pre_trained_model.trainable = True
for layer in pre_trained_model.layers[:-20]:
    layer.trainable = False

In [ ]:
model = Sequential([
    data_augmentation,
    pre_trained_model,
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(20, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2
)

In [ ]:
resnet = model.fit(
    train,
    epochs=50,
    validation_data=val,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
test_dir = '/kaggle/input/rice-pistachio-and-grapevine-leaf-classification/test/test'
test_filepaths = sorted([os.path.join(test_dir, fname) for fname in os.listdir(test_dir)])

def load_test(filepath):
    img = tf.io.read_file(filepath)
    img = tf.io.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, (256, 256))
    img = img / 255.0
    return img

test_ds = tf.data.Dataset.from_tensor_slices(test_filepaths)
test_ds = test_ds.map(load_test).batch(32)

predictions = model.predict(test_ds, verbose=1)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = [int_to_class[i] for i in predicted_class_indices]
file_names = [os.path.basename(p) for p in test_filepaths]
submission_df = pd.DataFrame({
    'ID': file_names,
    'TARGET': predicted_labels
})

submission_df.to_csv('submission.csv', index=False)
submission_df.head()